In [1]:
from dotenv import load_dotenv

load_dotenv("/home/ubuntu/work/edu-src-all/.env")
#load_dotenv()

True

In [2]:
!pip install -U openai langchain langchain-openai langchain-community langchain-experimental

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from langchain_openai import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_chat_planner,
    load_agent_executor
)

# 1. LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# 2. Planner
planner = load_chat_planner(llm)

# 3. Executor (Tool 없음)
executor = load_agent_executor(
    llm=llm,
    tools=[]
)

# 4. Agent
agent = PlanAndExecute(
    planner=planner,
    executor=executor
)


In [4]:
result = agent.run(
    "LLM 에이전트가 어떻게 작동하는지 4단계로 설명해줘."
)

print(result)

/tmp/ipykernel_74073/1760966496.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = agent.run(


LLM(대형 언어 모델)의 응답은 여러 방법으로 평가되고 개선됩니다. 일반적으로, 응답의 품질은 인간 평가자에 의해 주관적으로 평가되거나, 자동화된 메트릭(예: BLEU, ROUGE 등)을 사용하여 측정됩니다. 이러한 평가 결과는 모델의 성능을 분석하고, 어떤 부분에서 개선이 필요한지를 파악하는 데 도움을 줍니다. 또한, 사용자 피드백을 통해 모델은 지속적으로 학습하고 개선될 수 있습니다. 예를 들어, 사용자가 제공하는 긍정적 또는 부정적 피드백은 모델의 응답을 조정하는 데 사용되며, 이를 통해 더 나은 품질의 응답을 생성할 수 있도록 합니다. 이러한 과정은 반복적이며, 모델의 성능을 지속적으로 향상시키는 데 기여합니다.


In [5]:
for item in agent.stream("LLM 에이전트가 어떻게 작동하는지 4단계로 설명해줘."):
    print(item)

{'input': 'LLM 에이전트가 어떻게 작동하는지 4단계로 설명해줘.', 'output': 'LLM(대형 언어 모델)의 응답은 여러 방법으로 평가되고 개선됩니다. 일반적으로, 응답의 품질은 인간 평가자에 의해 주관적으로 평가되거나, 자동화된 메트릭(예: BLEU, ROUGE 등)을 사용하여 측정됩니다. 이러한 평가 결과는 모델의 성능을 분석하고, 어떤 부분에서 개선이 필요한지를 파악하는 데 도움을 줍니다. 또한, 사용자 피드백을 통해 모델은 지속적으로 학습하고 개선될 수 있습니다. 예를 들어, 사용자가 제공하는 긍정적 또는 부정적 피드백은 모델의 응답을 조정하는 데 사용되며, 이를 통해 더 나은 품질의 응답을 생성할 수 있도록 합니다. 이러한 과정은 반복적이며, 모델의 성능을 지속적으로 향상시키는 데 기여합니다.'}


In [6]:
from langchain_openai import ChatOpenAI
from langchain_experimental.plan_and_execute import load_chat_planner

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

planner = load_chat_planner(llm)

plan = planner.plan(
    {"input": "NVIDIA GPU 시장 동향을 분석하기 위해 필요한 단계들을 계획해줘."}
)

print(plan)


steps=[Step(value='NVIDIA GPU 시장에 대한 기본 정보 수집: NVIDIA의 제품 라인업, 주요 경쟁사, 시장 점유율 등을 조사한다.'), Step(value='최근 시장 동향 파악: 최신 뉴스, 보고서, 연구 자료를 통해 NVIDIA GPU의 판매 추세 및 기술 발전을 분석한다.'), Step(value='소비자 및 기업의 수요 분석: NVIDIA GPU를 사용하는 주요 산업 및 소비자 요구를 조사한다.'), Step(value='경쟁사 분석: AMD, Intel 등 주요 경쟁사의 GPU 제품 및 전략을 비교 분석한다.'), Step(value='시장 전망 및 예측: 수집한 데이터를 바탕으로 향후 NVIDIA GPU 시장의 성장 가능성 및 도전 과제를 예측한다.'), Step(value='분석 결과 정리 및 보고서 작성: 모든 분석 결과를 종합하여 보고서를 작성한다.\n\nGiven the above steps taken, please respond to the users original question. \n')]


In [7]:
from langchain_openai import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain.tools import BaseTool
import requests

# --- Tool ---
class WebSearchTool(BaseTool):
    name: str = "web_search"
    description: str = "Searches information on the web using DuckDuckGo"
    
    def _run(self, query: str):
        url = f"https://api.duckduckgo.com/?q={query}&format=json"
        res = requests.get(url).json()
        return res.get("Abstract", "No data found.")
    
    async def _arun(self, query: str):
        return self._run(query)


# --- LLM ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Planner & Executor ---
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools=[WebSearchTool()])

In [8]:
plan = planner.plan(
    {"input": "최근 NVIDIA 뉴스를 검색해서 요약해줘."}
)

print(plan)

steps=[Step(value='최근 NVIDIA와 관련된 주요 뉴스 기사를 검색한다.'), Step(value='검색된 기사에서 NVIDIA의 최신 제품, 기술 발전, 재무 성과 또는 시장 동향에 대한 정보를 추출한다.'), Step(value='추출한 정보를 간결하게 요약한다.'), Step(value='요약된 내용을 정리하여 제공한다.\nGiven the above steps taken, please respond to the users original question.\n')]


In [9]:

agent = PlanAndExecute(
    planner=planner,
    executor=executor,
)

# --- Run ---
result = agent.run("최근 NVIDIA 뉴스를 검색해서 요약해줘.")
print(result)


Agent stopped due to iteration limit or time limit.
